# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [1]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'
arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'

df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

def esri_object_to_df(in_esri_obj, esri_obj_fields, index_field=None):
    '''converts esri gdb table, feature class, feature layer, or SHP to pandas dataframe'''
    data_rows = []
    with arcpy.da.SearchCursor(in_esri_obj, esri_obj_fields) as cur:
        for row in cur:
            out_row = list(row)
            data_rows.append(out_row)

    out_df = pd.DataFrame(data_rows, index=index_field, columns=esri_obj_fields)
    return out_df

print([t for t in arcpy.ListTables()])


['rp_artexp_vmt', 'rp_artexp_econ', 'rp_artexp_eq', 'rp_artexp_mm', 'rp_artexp_sgr', 'rp_fwy_vmt', 'rp_fwy_cong', 'rp_fwy_mm', 'rp_fwy_econ', 'rp_fwy_frgt', 'rp_fwy_saf', 'rp_artsgr_sgr', 'cd_compactdev', 'cd_mixeduse', 'cd_houschoice', 'cd_naturpres', 'rp_artexp_cong', 'rp_artexp_frgt', 'rp_artexp_saf', 'cd_trnchoice', 'cd_existgasset', 'TEST_TABLE']


## View of most recent entries into master table

In [2]:
fields = ['proj_name','juris','proj_type','comm_type','time_created','user_email']
df_pmaster[fields].sort_values(by='time_created', ascending=False).head(10)

,proj_name,juris,proj_type,comm_type,time_created,user_email
273,Natoma Street Drainage Phase 3 Project,City of Folsom,Arterial State of Good Repair,Arterials & Suburban Corridors,2023-01-25 11:45:47.000000,rneves@folsom.ca.us
272,Lindhurst and North Beale Road,Yuba County,Arterial State of Good Repair,Small-Town Established Communities,2023-01-25 10:26:42.000000,dpeterson@co.yuba.ca.us
271,Pennington Road Complete Streets Rehabilitation,City of Live Oak,Arterial State of Good Repair,Small-Town Established Communities,2023-01-24 15:49:09.000000,srolls@rarcivil.com
270,Loma Rica and South Beale Rd,Yuba County,Arterial State of Good Repair,Ag,2023-01-24 14:41:39.000000,dpeterson@co.yuba.ca.us
269,Maintenance Paving Project,County of Sutter,Arterial State of Good Repair,Ag,2023-01-24 11:27:53.000001,LCarrillo@co.sutter.ca.us
268,8. Watt I-80 Station,SacRT,Community Design,Arterials & Suburban Corridors,2023-01-24 10:44:19.000001,gballard-rosa@sacog.org
267,7. Sac Valley Station,SacRT,Community Design,Urban core,2023-01-24 10:42:45.000001,gballard-rosa@sacog.org
266,6. Folsom Blvd,Sac County,Community Design,Arterials & Suburban Corridors,2023-01-24 10:40:39.000000,gballard-rosa@sacog.org
265,5. Arden Way,Sac County,Community Design,Arterials & Suburban Corridors,2023-01-24 10:38:35.000000,gballard-rosa@sacog.org
264,4. Setzer Tunnel,Sac City,Community Design,Established Communities,2023-01-24 10:37:21.000000,gballard-rosa@sacog.org


## Find UID values that are in any of the data tables but not in the master table

In [3]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master


## Show specific subreport data and for specific projects

In [18]:

subrpt_tbl = 'rp_artexp_cong'
project_uid = '926daf1e-174b-47b9-b5b1-4427e3ecad9c'

fields = [f.name for f in arcpy.ListFields(subrpt_tbl)]

#uncomment if you want to filter which fields appear
# fields = [f_uid, 'crash_cnt', 'crash_100mvmt'] # [f.name for f in arcpy.ListFields(subrpt_tbl)]

df_master_fields = [f_uid, 'time_created', 'user_email', 'len_mi', 'aadt']
df_subrpt = esri_object_to_df(subrpt_tbl, esri_obj_fields=fields, index_field=None)


df_jn = df_pmaster[df_master_fields].merge(df_subrpt, on=f_uid, suffixes=('','_y')) \
    .sort_values(by='time_created', ascending=False)

if project_uid:
    df = df_jn.loc[df_jn['project_uid'] == project_uid].head()
else:
    df = df_jn.head(10)
    
df

,project_uid,time_created,user_email,len_mi,aadt,OBJECTID,aadt_y,congspd_nb,congspd_sb,congspd_eb,...,jobs_base,jobs_future,du_base,du_future,congspd_wrst,congrat_wrst,lottr_ampk_wrst,lottr_midday_wrst,lottr_pmpk_wrst,lottr_wknd_wrst


In [42]:
df_jn.loc[df_jn.crash_100mvmt > 100].head()

,project_uid,time_created,user_email,len_mi,aadt,crash_cnt,crash_100mvmt
21,55160fbd-9617-4c09-a05e-da8819a2b71c,2022-12-06 13:47:37,jhong@sacog.org,3.670994,5.0,136,463089.843750
45,fbc41452-2ed7-44ce-b1d1-3563bad0f73c,2023-01-17 06:06:46,rbissegger@markthomas.com,1.861210,58600.0,36,130.566345
